# Linear Secret Sharing Schemes (a.k.a LSSS)

In the past two notebooks we have seen two secret sharing schemes, first one is additive secret sharing (where we defined sum and product) and Shamir secret sharing based on polynomial reconstruction. We didn't define product in this last one but don't worry we'll do soon.

Additive secret sharing and Shamir secret sharing are part of a group of secret sharing schemes called linear secret sharing shemes (LSSS) because they can be formalised using linear algebra. Before defining LSSS formally let's reformulate the additive secret sharing.

As we defined previously a $(t, n)$ threshold secret sharing scheme is a method for $n$ parties to carry shares of
a message s such that any $t$ of the them to reconstruct the message, but so that no $t − 1$
of them can easy do so. The threshold scheme is perfect if knowledge of $t − 1$ or fewer
shares provides no information regarding $s$.

Warning!!: In all the section all arithmetics will be modulo prime $p$ chosen. Sums, products and inverses are modulo $p$. To avoid repetition I will omit $\pmod{p}$ in all operations, but it is implicit.


## Linear reformulation of additive secret sharing

Remember that in this scheme the data holder splits the secret into $n$ random shares whose sum modulo $p$ is the original secret.

$$s = s_0+s_1+\cdots s_n=\sum_{i=0}^{i=n}s_i$$

For this case we define the identity matrix 

$$
\begin{align}
M = \mathbb{1}^{n\times n} =
\begin{bmatrix} 
    1 & 0 & \dots & 0\\
    0 & 1 & \dots & 0\\
    \vdots &  & \ddots & \vdots\\
    0 & 0 & \dots & 1\\
    \end{bmatrix}
\end{align}
$$ 

This is a matrix with diagonal ones and the rest zero. If we multipy any vector of size $n$ by this matrix we get the same vector. Actually this matrix lives in the space $\mathbb{F}^{n\times n}_{p}$ where $p$ is the agreed prime number.

We define a vector $\mathbf{v}$ of size $n$ and only ones in its entries

$$
\begin{align}
\textbf{v} = 
\begin{bmatrix} 
    1\\
    1\\
    \vdots \\
    1 \\
    \end{bmatrix}
\in \mathbb{F}^{n}_p
\end{align}
$$ 

A vector $\textbf{k}$ of random numbers and accomplishing property that when we sum all coefficcients and apply modulo we get the secret (same as the previous defined $s$ but remember we are reformulating the sscheme. 

$$
\begin{align}
\textbf{k} = 
\begin{bmatrix} 
    k_0\\
    k_1\\
    \vdots \\
    k_{n-1} \\
    \end{bmatrix}
\in \mathbb{F}^{n}_p
, s.t. \sum_{i=0}^{n-1}k_i = s
\end{align}
$$ 

With this defined we can get the vector of shares as:

$$
\begin{align}
\textbf{s} = 
\begin{bmatrix} 
    s_0\\
    s_1\\
    \vdots \\
    s_{n-1} \\
    \end{bmatrix}
=
\begin{bmatrix} 
    k_0\\
    k_1\\
    \vdots \\
    k_{n-1} \\
    \end{bmatrix}
=M\textbf{k}
\end{align}
$$ 

And recover the secret with the scalar product:

$$s = \textbf{v}^t \cdot \textbf{k} = k_0+k_1+\cdots+k_{n-1} $$

As always I've coded this. This time in a class called AdditiveLSSS

In [1]:
from smpc import AdditiveLSSS
from random import seed, randrange

seed(5)

bits = 32
parties = 5
scheme = AdditiveLSSS(n=parties, size=bits)

print(f"Chosen random prime {scheme.p}")
print(f"number of parties to split the secret is {scheme.n}")

# just not to put secret 10 we generate one random
secret = randrange(0, scheme.p)
print(f"Random secret chosen {secret}")

Chosen random prime 3244611641
number of parties to split the secret is 5
Random secret chosen 844054145


We get the vector k by setting the secret into the object

In [2]:
scheme.secret = secret
print(f"Generated k={scheme.k}")

Generated k=[ 511189398 1056934719 1985156919 1479108632 2300887759]


But every time we call the function generate we will generate and store the new shares of $\textbf{k}$

In [3]:
print("Generating new random shares")
scheme.GenerateK()
print(f"Generated k={scheme.k}")
print(f"whose sum modulo p is {int(sum(scheme.k)%scheme.p)} secret is {secret}\n")


Generating new random shares
Generated k=[2201153626 1523825202 2253577168 1077581322  277140109]
whose sum modulo p is 844054145 secret is 844054145



We regenerate again and show the shares each client is going to get

In [4]:
scheme.GenerateK()
print(f"Generated k={scheme.k}")
print(f"Dictonary of shares: {scheme.GetShares()}")
print(f"whose sum modulo p is {int(sum(scheme.k)%scheme.p)} secret is {secret}")

Generated k=[1987924596  464083540 2533014395 3214345152 2378521385]
Dictonary of shares: {0: [1987924596], 1: [464083540], 2: [2533014395], 3: [3214345152], 4: [2378521385]}
whose sum modulo p is 844054145 secret is 844054145


The recovery of the shares is coded in the same object

In [5]:
assert secret==scheme.RevealSecret(), "Something went wrong, secret incorrectly recovered"
print(f"The recovered secret is {scheme.RevealSecret()}")

The recovered secret is 844054145


## Linear reformulation of Shamir secret sharing

It is possible to formulate Shamir secret sharing in the LSSS. Let's define the polynomial and the parameters. The polynomial is 

$$
f(x)=a_0+a_1x+a_2x^2+\cdots+a_{t-1}x^{t-1}
$$

Where $a_0$ is the secret and we need at least $n=t$ shares to reconstruct perfectly the polynomial and thus get the secret.

The matrix $M$ in this LSSS is now the matrix of the evaluation of $1$, $x$, $x^2, \cdots, x^{t-1}$ for the values of $x$ from 1 until $n$.

$$
\begin{align}
M^{n\times t} = 
\begin{bmatrix} 
    1 & 1 & \dots & 1^{t-1}\\
    1 & 2 & \dots & 2^{t-1}\\
    \vdots &  &  & \vdots\\
    1 & n & \dots & n^{t-1}\\
    \end{bmatrix}
\end{align}
$$ 
i.e. this is a matrix $\mathbb{F}^{n\times t}$. Then, $\textbf{v}$ is 

$$
\begin{align}
\textbf{v} = 
\begin{bmatrix} 
    1\\
    0\\
    \vdots \\
    0 \\
    \end{bmatrix}
\in \mathbb{F}^t_p
\end{align}
$$

The vector $\textbf{k}$ must be generated as we did with Shamir shceme, this is $f(0)=a_0=s$, the rest of the coefficients are completely random. 

$$
\begin{align}
\textbf{k} = 
\begin{bmatrix} 
    a_0\\
    a_1\\
    \vdots \\
    a_{t-1} \\
    \end{bmatrix}
\in \mathbb{F}^{t}_p
, s.t. a_0 = s
\end{align}
$$ 

The shares are

$$
\begin{align}
\textbf{S}=M\textbf{k}=
\begin{bmatrix}
  a_{0}+a_{1}+a_2+\cdots+a_{t-1} \\
  a_{0}+2a_1+4a_2+\cdots+2^{t-1}a_{t-1}  \\
  \vdots \\
  a_{0}+na_1+n^2a_2+\cdots+n^{t-1}a_{t-1}
\end{bmatrix}
=
\begin{bmatrix}
  f(1) \\
  f(2)  \\
  \vdots \\
  f(n)
\end{bmatrix}
\end{align}
$$        
         
This is, the shares are the polynomial evaluated at points $x=1, 2, \cdots, n$. Obviously we are not going to send $f(0)$ to any party since it is our secret.

The secret is revealed as

$$s = \textbf{v}^t\textbf{k}=a_0=s$$

Following, the implementation in code

In [6]:
from smpc import ShamirLSSS

bits = 32
n = 5 #number of parties
t = 2 #Degree of polynomial, we need t+1 ponints to reconstruct

scheme = ShamirLSSS(n=n, t=t, size=bits)

print(f"Chosen random prime {scheme.p}")
print(f"number of parties to split the secret is {scheme.n}, threshold {t}")

# just not to put secret 10 we generate one random
secret = randrange(0, scheme.p)
print(f"Random secret chosen {secret}")

Chosen random prime 4095423053
number of parties to split the secret is 5, threshold 2
Random secret chosen 1554902337


In [7]:
scheme.secret = secret
print(f"Generated k={scheme.k}")

Generated k=[1554902337 3085772251  179794653]


In [8]:
print(f"Generated k={scheme.k}")
print(f"Dictonary of shares: {scheme.GetShares()}")

Generated k=[1554902337 3085772251  179794653]
Dictonary of shares: {0: [4820469241], 1: [8445625451], 2: [12430370967], 3: [16774705789], 4: [21478629917]}


In [9]:
assert secret==scheme.RevealSecret(), "Something went wrong, secret incorrectly recovered"
print(f"The recovered secret is {scheme.RevealSecret()}")

The recovered secret is 1554902337


## Linear reformulation of Replicated Secret Sharing

In [10]:
from smpc import ReplicatedLSSS

bits = 32

scheme = ReplicatedLSSS(size=bits)

print(f"Chosen random prime {scheme.p}")
print(f"number of parties to split the secret is {scheme.n}, threshold {t}")

# just not to put secret 10 we generate one random
secret = randrange(0, scheme.p)
print(f"Random secret chosen {secret}")

Chosen random prime 2702220557
number of parties to split the secret is 3, threshold 2
Random secret chosen 462141184


In [11]:
scheme.secret = secret
print(f"Generated k={scheme.k}")
print(f"Dictonary of shares: {scheme.GetShares()}")

Generated k=[2343301041  821108895 2702172362]
Dictonary of shares: {0: [821108895, 2702172362], 1: [2343301041, 2702172362], 2: [2343301041, 821108895]}


See that now each party gets two shares and some of them are repated. 

In [12]:
print(scheme.GetShares()[0][0]==scheme.GetShares()[2][1])
print(scheme.GetShares()[0][1]==scheme.GetShares()[1][1])

True
True


Again we can regenerate the K's and recompute the different shares

In [13]:
scheme.GenerateK()
scheme.GetShares()

{0: [1845420870, 1256403922],
 1: [62536949, 1256403922],
 2: [62536949, 1845420870]}

In [14]:
assert secret==scheme.RevealSecret(), "Something went wrong, secret incorrectly recovered"
print(f"The recovered secret is {scheme.RevealSecret()}")

The recovered secret is 462141184
